### Replace ECFP & Padel discriptor -> Mol2vec 
### for separate protein Agonist & Antagonist training using Autogluon training

In [2]:
# run the cell first for all setup

import re
from re import search
import os, sys
from os import walk
import traceback
import subprocess
import pandas as pd

import time
import math
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split
from autogluon.tabular import FeatureMetadata
from autogluon.tabular import TabularDataset, TabularPredictor


os.environ["CUDA_LAUNCH_BLOCKING"]="1"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"             # CPU only
os.environ["RAY_OBJECT_STORE_ALLOW_SLOW_STORAGE"] = "1"

def column_del(name_columnD,df_temp_del, column_drop_list):
    for i, columndel in enumerate (column_drop_list):
        try:
            df_temp_del=df_temp_del.drop(columns=[columndel], axis=1)
        except:    
            print (' ')
    return (df_temp_del)

def dataset_split_trainvalidtest (dataload, trainfilein):
    trainprotein=re.sub('\.csv','',trainfilein)
    traindata_split_filename=trainprotein+'_train.csv'
    validdata_split_filename=trainprotein+'_valid.csv'
    trainsize=0.8 # 0.85 or 0.7
    traindata, validdata = train_test_split(dataload, train_size=trainsize) 
    os.chdir(workpath)
    traindata.to_csv(traindata_split_filename,index=False,encoding='utf-8')
    validdata.to_csv(validdata_split_filename,index=False,encoding='utf-8')
    print ('Split dataset : ', trainfilein, ' -> ', traindata_split_filename,' & ', validdata_split_filename)
    return (traindata, validdata)

def AUTOGLUON_train(workpath,trainfilein,AIlabel_problem_eval,trainname):
    starttime1=time.time()
    os.chdir(workpath)
    AIlabel,problem_type,eval_metric,AgonistAnagonist_label=AIlabel_problem_eval
    trainname1=trainname+'_'
    trainname2=re.sub('GPCR_to_train','',trainname) 
    dataload=TabularDataset(trainfilein)
    dataload.replace([np.inf, -np.inf], np.nan, inplace=True) ###

    trainproteinname=re.sub(trainname1,'',trainfilein) 
    trainproteinname=re.sub(trainname,'',trainproteinname) 
    trainproteinname=re.sub('\.csv','',trainproteinname) 
    ###
    dataload=column_del(name_columnD=trainfilein,df_temp_del=dataload, column_drop_list=['Protein_name', 'ChEMBL ID', 'Species','EfficacyValue_percentage','level_0','Unnamed: 0','index','ECFP', 'Smiles', 'padel1d2d','mol2vec'])
    ###
    shape_x0,shape_y0=dataload.shape
    AGtrain=''
    if shape_x0 >8:
        AGtrain='True'        
        traindata,validdata=dataset_split_trainvalidtest(dataload,trainfilein)
        all_x,all_y=dataload.shape
        shape_x,shape_y =traindata.shape
        shape_x2,shape_y2 =validdata.shape
        timelimit = 'YES'  # 'No' 


        traintimelimit=1.8*60*60 
        
        timestamp=time.strftime("%m%d",time.localtime(time.time()))
        print ('\n Now training: ',trainproteinname,' , training data size: ', shape_x, shape_y,' , test data size: ',shape_x2, shape_y2 , ' , \n Train time: ', (traintimelimit/60) ,' mins  at', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())) ,'\n\n\n')
        if timelimit == 'YES':
            if traintimelimit>60:
                float2time=int(traintimelimit/60)     #"{:.2f}".format(traintimelimit/60)
                ML_path = 'AutogluonTRAIN'+timestamp+'_'+trainname2+'_'+trainproteinname+'_'+str(float2time)+'min'
            else:
                float2time=int(traintimelimit)
                ML_path = 'AutogluonTRAIN'+timestamp+'_'+trainname2+'_'+trainproteinname+'_'+str(float2time)+'sec'
        else:
            timestamp=time.strftime("%Y%m%d_%H%M",time.localtime(time.time()))
            ML_path = 'AutogluonTRAIN'+timestamp+'_'+trainname2+'_'+trainproteinname+'_nolimit'
        
        AGtrain=''
        AGtrain_time=0
        while AGtrain=='' : 
            try:
                predictor = TabularPredictor(label=AIlabel, problem_type=problem_type, eval_metric=eval_metric,path=ML_path, verbosity=3).fit(train_data=traindata, auto_stack=True, time_limit=traintimelimit, excluded_model_types=['KNN'])  
                print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Trained ', trainproteinname ,(time.time()-starttime1)/60, ' mins ', '\n\n' )
                AGtrain='True'
            except Exception as e:
                print('\n *** Error: ',e,' \n')
                print ('\n ***  REDO Autogluon train : ',AGtrain_time,' for the file: ', trainfilein)
                print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n')
                AGtrain_time=AGtrain_time+1
                traintimelimit=traintimelimit+1
                AGtrain=''
            if AGtrain_time > 7:
                AGtrain='False'
                leaderboard=pd.DataFrame()
                predictor=pd.DataFrame()
                print ('\n ***  Fail Autogluon train : ',AGtrain_time,' for the file: ', trainfilein,' -- NG -- ')
            
        if AGtrain=='True':
            try:
                leaderboard=predictor.leaderboard(validdata, extra_metrics=['mse','mae','pearsonr','rmse'], silent=True)
                leaderboard=leaderboard.sort_values(by=['score_val'],ascending=False)
                leaderboard=leaderboard.reset_index(drop=True)
                print ('\n\n',leaderboard,'\n Best model: ', predictor.get_model_best())
                leaderboardfile=ML_path+'/leaderboard_'+trainproteinname+'_size'+str(shape_x)+'.csv'
                os.chdir(workpath)
                leaderboard.to_csv(leaderboardfile,index=False,encoding='utf-8')
                print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n')
            except Exception as e:
                print('\n *** Error: ',e,' \n')
                print (' * CANNOT do evaluations -> skip', trainfilein,'\n')
                AGtrain='False'
                leaderboard=pd.DataFrame()
                predictor=pd.DataFrame()
    else:
        AGtrain='False'
        ML_path=''
        leaderboard=pd.DataFrame()
        predictor=pd.DataFrame()
        traintimelimit=0
        shape_x=int(shape_x0)
        shape_x2=0
        traintimelimit=0
        trainprotein=re.sub(r'.csv','',trainfilein)
        rename_trainfilein=trainprotein+'_smalldataset.csv'
        cmd='cp '+trainfilein+' '+rename_trainfilein
        print ('\n\n small dataset -> copy file: ', trainfilein, rename_trainfilein, '\n  ', cmd)
        os.chdir(workpath)
        subprocess.run(cmd, shell=True) #, capture_output=True)
        print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n')
    return(AGtrain,leaderboard,predictor,ML_path,shape_x0,shape_x,shape_x2,traintimelimit)


def HUMANTRAINALLconcat(workpath,substr,trainname):
    os.chdir(workpath)
    k=0
    l=0
    alltrainconcat_file=trainname+'_humanalltrainconcat.csv'
    filenames = next(walk(workpath), (None, None, []))[2]
    concat_file_list=[]
    for i, fff in enumerate (filenames):
        if re.search('all', fff):
            continue
        if re.search('_HUMAN_train.csv', fff):
            concat_file_list.append(fff)
    sorted(concat_file_list)
    print ('concat_file_list: ',concat_file_list, len(concat_file_list))
    df_train_all=pd.DataFrame()
    for k, to_concat_file in enumerate (concat_file_list):
        os.chdir(workpath)
        to_concat_file_ori=to_concat_file ##re.sub('_train.csv','',to_concat_file) 
        df_read=pd.read_csv(to_concat_file_ori, low_memory=False) # concat _HUMAN_train.csv files
        print ('-------------- concat file: (',l,') ',to_concat_file_ori,len(df_read),' to ', alltrainconcat_file, len(df_train_all))
        if l==0:
            df_train_all=df_read
            l=l+1
            df_read_0=df_read
            column_labels=df_read_0.columns
        else:
            try:
                df_train_all=pd.concat([df_train_all,df_read], axis=0, ignore_index=True)
                l=l+1
            except:
                print ('skip file: ', to_concat_file_ori)

    os.chdir(workpath)
    print ('Total concat proteins: ', l, '\n concat_file_list: ',concat_file_list, len(concat_file_list))
    df_train_all.to_csv(alltrainconcat_file,index=False,encoding='utf-8')
    print ('Columns: ', df_train_all.columns)
    return (alltrainconcat_file)


def toTRAINfiles(workpath,trainnum,filenames,df_protein_train,substr,train_result_summary_file,AIlabel_problem_eval):
    os.chdir(workpath)
    ec50_label,problem_type,eval_metric,AgonistAnagonist_label= AIlabel_problem_eval
    print ('To train files: ', filenames, len(filenames))
    for i, trainfilein in enumerate (filenames):
        os.chdir(workpath)
        print (' ------------------------------ ',trainnum, trainfilein ,' ------------------------------ ')
        if re.search(substr[0], trainfilein):
            if re.search(substr[3], trainfilein):
                trainname=substr[3]
            elif re.search(substr[4], trainfilein):
                trainname=substr[4]
            elif re.search(substr[5], trainfilein):
                trainname=substr[5]
            trainname1=trainname+'_'
            trainproteinname=re.sub(trainname1,'',trainfilein)
            trainproteinname=re.sub('\.csv','',trainproteinname)
        df_protein_train.at[trainnum,'trainnum']="{:.0f}".format(trainnum)
        df_protein_train.at[trainnum,'Name']=trainproteinname
        trainname2=re.sub('GPCR_to_train','',trainname)
        df_protein_train.at[trainnum,'trainname']=trainname2
        df_protein_train.at[trainnum,'Time']=str(time.strftime("%Y%m%d_%H%M",time.localtime(time.time())))

        df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')
        print ('\n Run AutoGluon: (',trainnum,')', trainfilein,' - ' ,trainproteinname)
        print('\n',time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n')

        AGtrain,leaderboard,predictor,ML_path,shape_x0,shape_x,shape_x2,traintimelimit=AUTOGLUON_train(workpath,trainfilein,AIlabel_problem_eval,trainname)

        if AGtrain=='True':
            
            try:
                print (trainfilein, ' size ',shape_x,'\n BEST model: ',predictor.get_model_best(),', score: ', "{:.2f}".format(leaderboard.at[0,'score_val'] ,' \n'))
                df_protein_train.at[trainnum,'ALL_size']="{:.0f}".format(shape_x0)
                df_protein_train.at[trainnum,'TRAIN_size']="{:.0f}".format(shape_x)
                df_protein_train.at[trainnum,'TEST_size']="{:.0f}".format(shape_x2)
                df_protein_train.at[trainnum,'BEST_model']=predictor.get_model_best()
                df_protein_train.at[trainnum,'score_val']="{:.5f}".format(abs(leaderboard.at[0,'score_val']))                
                df_protein_train.at[trainnum,'mean_squared_error']="{:.5f}".format(abs(leaderboard.at[0,'mean_squared_error']))
                df_protein_train.at[trainnum,'pearsonr']="{:.5f}".format(leaderboard.at[0,'pearsonr'])
                df_protein_train.at[trainnum,'mean_absolute_error']="{:.5f}".format(abs(leaderboard.at[0,'mean_absolute_error']))
                df_protein_train.at[trainnum,'root_mean_squared_error']="{:.5f}".format(abs(leaderboard.at[0,'root_mean_squared_error']))
                df_protein_train.at[trainnum,'score_test']="{:.5f}".format(leaderboard.at[0,'score_test'])
                df_protein_train.at[trainnum,'score_val']="{:.5f}".format(leaderboard.at[0,'score_val'])
                df_protein_train.at[trainnum,'traintimelimit_mins']="{:.5f}".format(traintimelimit/60)
            except:
                df_protein_train.at[trainnum,'ALL_size']="{:.0f}".format(shape_x0)
                df_protein_train.at[trainnum,'TRAIN_size']="{:.0f}".format(shape_x)
                df_protein_train.at[trainnum,'BEST_model']='skip'
                            
        else:
            print ('\n skip \n\n', trainnum, trainfilein)
            df_protein_train.at[trainnum,'ALL_size']="{:.0f}".format(shape_x0)
            df_protein_train.at[trainnum,'TRAIN_size']="{:.0f}".format(shape_x)
            df_protein_train.at[trainnum,'BEST_model']='skip'
        df_protein_train.at[trainnum,'File_Name']=str(trainfilein)
        trainnum=trainnum+1
        df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')
        print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), ' done training: ', trainnum,trainfilein)

    os.chdir(workpath)
    df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')
    print ('\n ********************************* \n',df_protein_train,'\n ********************************* \n')
    print('\n', time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())),  ' done training proteins \n', filenames, trainnum, '\n\n\n\n\n\n\n')
    return (df_protein_train,trainnum)


def TRAINevaluateDATA(workpath,predictor,ML_path,fileevaluation,AG_ATG,AIlabel_problem_eval,df_TRAINevaluate,trainnum):  
    os.chdir(workpath)
    print ('=== ', fileevaluation, ' ===')
    AIlabel=AIlabel_problem_eval[0]
    AgonistAnagonist_label=AIlabel_problem_eval[3]

    
    try:
        test_data0=TabularDataset(fileevaluation)  #validdata

        ML_path_split=re.split('\/',ML_path)
        ML_path_AgonistAntagonist_test=ML_path_split[-1]
        
        if AG_ATG=='AG':
            test_data=test_data0.loc[test_data0[AgonistAnagonist_label]==1]
            AgonistAntagonist_test='Agonist'
        elif AG_ATG=='ATG':
            test_data=test_data0.loc[test_data0[AgonistAnagonist_label]==-1]
            AgonistAntagonist_test='Antagonist'
        else:
            test_data=test_data0.copy()
            AgonistAntagonist_test='AgonistAntagonist'

        if re.search('HUMAN',ML_path_AgonistAntagonist_test):
            species='HUMAN'
        elif re.search('HUMNA',ML_path_AgonistAntagonist_test):
            species='HUMAN'
        elif re.search('human',ML_path_AgonistAntagonist_test):
            species='HUMAN'
        elif re.search('INTER',ML_path_AgonistAntagonist_test):
            species='INTERSPECIES'
        elif re.search('all',ML_path_AgonistAntagonist_test):
            species='INTERSPECIES'
        else:
            species='INTERSPECIES_check'
            print('*** check ML_path  ***', ML_path_AgonistAntagonist_test,species)

        ### 
        test_data=test_data.dropna() ## copy()  ## clean data for na
        test_data=test_data.reset_index(drop=True)

        df_TRAINevaluate.at[trainnum,'SPECIES']=species
        df_TRAINevaluate.at[trainnum,'AgonistAntagonist_test']=AgonistAntagonist_test
        df_TRAINevaluate.at[trainnum,'test_data size']="{:.0f}".format(len(test_data))
        
        try:
            test_evaluate=predictor.evaluate(test_data)
            print (' \n ++++++++++ \n ', test_evaluate,' \n ++++++++++ \n')
            df_TRAINevaluate.at[trainnum,'mean_squared_error']=abs(test_evaluate['mean_squared_error'])
            df_TRAINevaluate.at[trainnum,'pearsonr']=test_evaluate['pearsonr']
            df_TRAINevaluate.at[trainnum,'mean_absolute_error']=abs(test_evaluate['mean_absolute_error'])
            df_TRAINevaluate.at[trainnum,'root_mean_squared_error']=abs(test_evaluate['root_mean_squared_error'])
            df_TRAINevaluate.at[trainnum,'r2']="{:.3f}".format(abs(test_evaluate['r2']))
            df_TRAINevaluate.at[trainnum,'median_absolute_error']=abs(test_evaluate['median_absolute_error'])
        except:
            print ('No test evaluate')
    except:
        print ('*** wrong data ***')
    return (df_TRAINevaluate,trainnum)

def proteinfiles_evaluation_test(workpath,predictor,ML_path,m,proteinpath,proteinfiles,evaluation_type,AG_ATG,AIlabel_problem_eval,df_TRAINevaluate,trainnum,outputfile,index_vectors_len):
    starttime1=time.time()
    peotein_index_len,padel_index_len,ECFP_index_len=index_vectors_len
    for i,ppp_evaluate in enumerate(proteinfiles):
        print ('\n\n ----------- \n',m, ML_path,'\n', i, ppp_evaluate, '\n ----------- ')
        fileevaluation=proteinpath+'/'+ppp_evaluate
        df_TRAINevaluate.at[trainnum,'ML_fullpath']=ML_path
        df_TRAINevaluate.at[trainnum,'peotein_index']="{:.0f}".format(peotein_index_len)
        df_TRAINevaluate.at[trainnum,'padel_index']="{:.0f}".format(padel_index_len)
        df_TRAINevaluate.at[trainnum,'ECFP_index']="{:.0f}".format(ECFP_index_len)
        df_TRAINevaluate.at[trainnum,'file_evaluate']=ppp_evaluate
        proteinname=re.sub('GPCR_to_trainAgonistAntagonist_','',ppp_evaluate)
        proteinname=re.sub('GPCR_to_trainAgonist_','',proteinname)
        proteinname=re.sub('GPCR_to_trainAntagonist_','',proteinname)
        proteinname=re.sub('GPCR_to_test_','',proteinname)
        proteinname=re.sub('_valid.csv','',proteinname)
        proteinname=re.sub('_train.csv','',proteinname)
        proteinname=re.sub('_smalldataset.csv','',proteinname)
        df_TRAINevaluate.at[trainnum,'Name']=proteinname
        df_TRAINevaluate.at[trainnum,'evaluate']=evaluation_type
        df_TRAINevaluate.at[trainnum,'AG_ATG']=AG_ATG
        df_TRAINevaluate,trainnum=TRAINevaluateDATA(workpath,predictor,ML_path,fileevaluation,AG_ATG,AIlabel_problem_eval,df_TRAINevaluate,trainnum)
        trainnum=trainnum+1
        os.chdir(workpath)
        df_TRAINevaluate.to_csv(outputfile,index=False,encoding='utf-8')
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n sub-total = ', (time.time()-starttime1)/3600, ' hrs \n +++++++++++++ \n'  )
    return(df_TRAINevaluate,trainnum)

homepath=str(os.getcwd())






### CODE: Main


In [ ]:
workpath=homepath+'/Human_AgonistAntagonist/GPCR_to_test_Mol2Vec/seperatePROTEIN_Mol2Vec'


timestamp=time.strftime("%Y%m%d%H%M",time.localtime(time.time()))
print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())))
starttime=time.time()

ec50_label='ActivityValue_log_EC50'
AgonistAnagonist_label='AgonistP1AnagonistN1_label'
problem_type ='regression'  
eval_metric='mean_squared_error'

AIlabel_problem_eval=[ec50_label,problem_type,eval_metric,AgonistAnagonist_label]
substr=['GPCR_to_train','_train.csv','_valid.csv','GPCR_to_trainAgonistAntagonist','GPCR_to_trainAgonist','GPCR_to_trainAntagonist']

train_result_summary_file='protein_train_leaderboard_evaluate_sklearn_'+str(timestamp)+'.csv'


trainname=substr[3]
os.chdir(workpath)

pwd_path=str(os.getcwd())
print ('Now path = ', pwd_path)

df_protein_train=pd.DataFrame()

trainnum=0

########################################

filenames = ['GPCR_to_trainAgonistAntagonist_humanalltrainconcat_selefeatureTOP200.csv']
df_protein_train,trainnum = toTRAINfiles (workpath,trainnum,filenames,df_protein_train,substr,train_result_summary_file,AIlabel_problem_eval)


os.chdir(workpath)
df_protein_train.to_csv(train_result_summary_file,index=False,encoding='utf-8')

##########################################

if (time.time()-starttime)/3600 > 1 :
    print('\n',time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/3600, ' hrs \n' )
elif (time.time()-starttime)/60 > 1 :
    print('\n',time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/60, ' mins \n' )
else:
    print('\n',time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime), ' secs \n' )




##### result data:

In [3]:
workpath_home=homepath+'/GPCR_to_test_Mol2Vec'
os.chdir(workpath_home)
df_TRAINevaluate=pd.read_csv('TRAINevaluate_proteins_valid_smalltest_separateAGATG202312140117.csv')
df_TRAINevaluate

,ML_fullpath,file_evaluate,Name,evaluate,AG_ATG,SPECIES,AgonistAntagonist_test,test_data size,mean_squared_error,pearsonr,mean_absolute_error,root_mean_squared_error,r2,median_absolute_error
0,AutogluonTRAIN1213_AgonistAntagonist_humanallt...,GPCR_to_trainAgonistAntagonist_5HT1A_HUMAN_val...,5HT1A_HUMAN,human_valid,AGATG,HUMAN,AgonistAntagonist,170,0.212253,0.929292,0.299672,0.460709,0.863,0.173817
1,AutogluonTRAIN1213_AgonistAntagonist_humanallt...,GPCR_to_trainAgonistAntagonist_5HT1B_HUMAN_val...,5HT1B_HUMAN,human_valid,AGATG,HUMAN,AgonistAntagonist,33,0.213953,0.912275,0.324998,0.462550,0.823,0.266039
2,AutogluonTRAIN1213_AgonistAntagonist_humanallt...,GPCR_to_trainAgonistAntagonist_5HT1D_HUMAN_val...,5HT1D_HUMAN,human_valid,AGATG,HUMAN,AgonistAntagonist,50,0.213188,0.926207,0.340942,0.461723,0.833,0.284652
3,AutogluonTRAIN1213_AgonistAntagonist_humanallt...,GPCR_to_trainAgonistAntagonist_5HT1E_HUMAN_val...,5HT1E_HUMAN,human_valid,AGATG,HUMAN,AgonistAntagonist,8,0.050118,0.865125,0.182479,0.223870,0.495,0.185851
4,AutogluonTRAIN1213_AgonistAntagonist_humanallt...,GPCR_to_trainAgonistAntagonist_5HT2A_HUMAN_val...,5HT2A_HUMAN,human_valid,AGATG,HUMAN,AgonistAntagonist,107,0.300666,0.848074,0.356099,0.548330,0.710,0.227334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,AutogluonTRAIN1213_AgonistAntagonist_humanallt...,GPCR_to_trainAgonistAntagonist_SCTR_HUMAN_smal...,SCTR_HUMAN,human_small,AGATG,HUMAN,AgonistAntagonist,4,4.376836,NaN,2.085426,2.092089,0.000,2.085426
428,AutogluonTRAIN1213_AgonistAntagonist_humanallt...,GPCR_to_trainAgonistAntagonist_SSR1_HUMAN_smal...,SSR1_HUMAN,human_small,AGATG,HUMAN,AgonistAntagonist,8,1.752175,0.358090,1.132901,1.323697,2.303,1.234913
429,AutogluonTRAIN1213_AgonistAntagonist_humanallt...,GPCR_to_trainAgonistAntagonist_SSR4_HUMAN_smal...,SSR4_HUMAN,human_small,AGATG,HUMAN,AgonistAntagonist,8,4.779861,0.460514,2.148029,2.186289,25.462,2.256629
430,AutogluonTRAIN1213_AgonistAntagonist_humanallt...,GPCR_to_trainAgonistAntagonist_T2R31_HUMAN_sma...,T2R31_HUMAN,human_small,AGATG,HUMAN,AgonistAntagonist,2,1.357734,NaN,1.165219,1.165219,0.000,1.165219
